In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import igraph as ig
import networkx as nx
import graphistry

In [2]:
data_path = '/public/npatil4/networks/data/combined/'

file_hour = data_path + 'hourwise.txt'
data_hour = pd.read_csv(file_hour,
                        sep='|')

In [3]:
def data_limited(df):
    
    mask = (
        (df.pick_long.between(-75,-73).values) & 
        (df.pick_lat.between(40,41).values) &
        (df.drop_long.between(-75,-73).values) & 
        (df.drop_lat.between(40,41).values) &
        (df.start!=data_year.stop)
        )
    
    return(mask)

def data_limited2(df):
    
    mask = (
        (df.pick_long.between(-74.02,-73.92).values) & 
        (df.pick_lat.between(40.70,40.85).values) &
        (df.drop_long.between(-74.02,-73.92).values) & 
        (df.drop_lat.between(40.70,40.85).values)
        )
    
    return(mask)

#### Mask : Geo-bound

In [4]:
mask = (data_limited2(data_hour))
print 'Routes : ',sum(mask)
print 'Trips : ',data_hour.trips[mask].sum()

Routes :  122803015
Trips :  125169721


#### Generate graph

data_nwk =nx.DiGraph()
data_nwk = nx.from_pandas_dataframe(df = data_hour[mask],
                                    source='start',
                                    target='stop',
                                    edge_attr=['trips'],
                                    create_using=data_nwk)

#### Map geo-cordinates to census-tract

In [5]:
graph_census_tracts_2 = pd.read_csv('../data/graph_censusTract_full.txt',sep='|')
graph_census_tracts_2.columns = ['start_tract']+[str(x) for x in graph_census_tracts_2.columns[1:]]
for col in graph_census_tracts_2.columns:
    graph_census_tracts_2[col] = graph_census_tracts_2[col].astype('str')
    
    
graph_census_tracts_2['CENTLAT'] = graph_census_tracts_2.CENTLAT.astype('float')
graph_census_tracts_2['CENTLON'] = graph_census_tracts_2.CENTLON.astype('float')

In [6]:
data_use = data_hour[mask].merge(graph_census_tracts_2[
                            ['CENTLAT','CENTLON','start_tract','GEOID']],
                left_on='start',right_on='start_tract').merge(
                            graph_census_tracts_2[
                            ['CENTLAT','CENTLON','start_tract','GEOID']],
                left_on='stop',right_on='start_tract')

In [9]:
data_use['date'] = pd.to_datetime(data_use.date)
data_use['weekDay'] = data_use.date.dt.weekday

In [10]:
columns = ['trips','dist_trip',
           'CENTLAT_x','CENTLON_x',
           'start_tract_x','GEOID_x',
           'CENTLAT_y','CENTLON_y',
           'start_tract_y','GEOID_y',
           'pick_lat','pick_long',
           'drop_lat','drop_long',
           'date']

data_use=data_use[columns]

data_use.columns = ['trips','dist_trip',
                    'startLat','startLong',
                    'start_original','startGeoId',
                    'stopLat','stopLong',
                    'stop_original','stopGeoId',
                    'pick_lat','pick_long',
                    'drop_lat','drop_long',
                    'date']


data_use2 = data_use.groupby(['startGeoId','stopGeoId',
                             'startLat','startLong',
                             'stopLat','stopLong',
                             'date'],
                            as_index=False)['trips'].sum()

data_use3 = data_use.groupby(['start_original','stop_original',
                             'pick_lat','pick_long',
                             'drop_lat','drop_long',
                             'date'],
                            as_index=False)['trips'].sum()

In [13]:
data_use3.shape

(122803015, 8)

In [52]:
data_use3['day'] = data_use3.date.dt.hour
data_use3['hour'] = data_use3.date.dt.hour
data_use3['weekDay'] = data_use3.date.dt.weekday_name

In [53]:
data_hour_grouped_out = data_use3.groupby(['start_original',
                             'pick_lat','pick_long',
                             'day','hour'],
                             as_index=False)['trips'].sum()
data_hour_grouped_in = data_use3.groupby(['stop_original',
                             'drop_lat','drop_long',
                             'hour'],
                             as_index=False)['trips'].sum()

data_hour_grouped_out = data_use3.groupby(['start_original',
                             'pick_lat','pick_long','hour'],
                             as_index=False)['trips'].mean()
data_hour_grouped_in = data_use3.groupby(['stop_original',
                             'drop_lat','drop_long','hour'],
                             as_index=False)['trips'].mean()

In [50]:
data_use2['day'] = data_use2.date.dt.dayofyear
data_use2['hour'] = data_use2.date.dt.hour
data_use2['weekDay'] = data_use.date.dt.weekday_name

data_hour_grouped_out = data_use2.groupby(['startGeoId',
                             'startLat','startLong',
                             'day','hour'],
                             as_index=False)['trips'].sum()

data_hour_grouped_out = data_use2.groupby(['startGeoId',
                             'startLat','startLong',
                             'hour'],
                             as_index=False)['trips'].mean()

data_hour_grouped_in = data_use2.groupby(['stopGeoId',
                             'stopLat','stopLong',
                             'day','hour'],
                             as_index=False)['trips'].sum()
data_hour_grouped_in = data_use2.groupby(['stopGeoId',
                             'stopLat','stopLong',
                             'hour'],
                             as_index=False)['trips'].mean()

In [54]:
data_hour_grouped_out.to_csv('../data/data_use_outgoing_original.txt',index=False)
data_hour_grouped_in.to_csv('../data/data_use_incoming_original.txt',index=False)